# Introduction

This is a quick introduction to the tradebook module. The tradebook is just a log of trades that shows your positions and values based on the trades you have done and provide you a few helper methods. This provides a flexible approach for simulating trades based on an event based system or a system where you iterate through each line as a separate observation.

Caveat
--------
**This is not an orderbook.** All trades are assumed to be executed and its kept as simple as possible.

## Initialize a tradebook

In [ ]:
import pandas as pd
from fastbt.tradebook import TradeBook
tb = TradeBook()

### Add some trades

To add trades to a tradebook, you need 5 mandatory parameters

 * timestamp - could be a string or an id; but datetime or pandas timestamp preferred
 * symbol - the security symbol or asset code
 * price - float/number
 * qty - float/number
 * order - **B for BUY and S for SELL**
 
 
Just use the `add_trade` method to add trades. Its the only method to add trades. You can include any other keyword arguments to add additional information to trades.

 

In [ ]:
# Let's add a few trades
tb.add_trade(pd.to_datetime('2019-02-01'), 'AAA', 100, 100, 'B')
tb.add_trade(pd.to_datetime('2019-02-02'), 'AAA', 102, 100, 'B')
tb.add_trade(pd.to_datetime('2019-02-02'), 'BBB', 1000, 15, 'S')

### Get some information

Internally all data is represented as dictionaries

Use
 * `tb.positions` to get the positions for all stocks
 * `tb.values` to get the values
 * `tb.trades` for trades
 
To get these details for a single stock use, `tb.positions.get()`
 

In [ ]:
# Print tradebook summary
tb # Shows that you have made 3 trades and 2 of the positions are still open

In [ ]:
# Get positions
tb.positions # negative indicates short position

In [ ]:
# Get position for a particular stock
print(tb.positions.get('AAA'))

In [ ]:
# Get the current value
tb.values # Negative values indicate cash outflow

In [ ]:
# Get the trades
tb.trades.get('AAA')

In [ ]:
# Get all your trades
tb.all_trades

In [ ]:
# A few helper methods
print(tb.o) # Number of open positions
print(tb.l) # Number of long positions
print(tb.s) # Number of short positions

### Something to look out for 

 * A position of zero indicates that all trades are settled
 * A positive position indicates holdings
 * A negative position indicates short selling
 * Conversely, a positive value indicates money received from short selling and a negative value indicates cash outflow for buying holding
 * If all positions are zero, then the corresponding values indicate profit or loss
 * Trades are represented as a dictionary with keys being the symbol and values being the list of all trades. So to get the first trade, use `tb.trades[symbol][0]`
 
Let's try out by closing all existing positions

In [ ]:
tb.positions, tb.values

In [ ]:
# Close existing positions
tb.add_trade(pd.to_datetime('2019-03-05'), 'AAA', 105, 200, 'S', info='exit')
tb.add_trade(pd.to_datetime('2019-03-05'), 'BBB', 1010, 15, 'B', info='exit')
tb.positions, tb.values

> You could now see that both the positions are closed but you got a profit on AAA and a loss on BBB

In [ ]:
# Summing up total profit
print(tb)
tb.values.values()

> And you could nicely load them up in a dataframe and see your additional info column added

In [ ]:
pd.DataFrame(tb.all_trades).sort_values(by='ts')

# Creating a strategy

Let's create a simple strategy for bitcoin and let's see how it works. This is a long only strategy

> **ENTER** when 7 day simple moving average (SMA) is greater than 30 day SMA and **EXIT** when 7 day SMA is less than 30 day SMA

Other info
-----------

* Invest $10000 for each trade
* Hold only one position at a single time (BUY only, no reversals)
* If you are already holding a position, check for the exit rule
* SMA is calculated on OPEN price and its assumed that you buy and sell at the open price

The sample file already has the columns sma7 and sma30

In [ ]:
df = pd.read_csv('data/BTC.csv', parse_dates=['date'])

In [ ]:
# We would be using standard Python csv library

import csv
filename = 'data/BTC.csv' # File available in data directory
btc = TradeBook()
capital = 10000 # this is fixed
with open(filename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Convert to floats since by default csv reads everything as string
        sma7 = float(row['sma7']) + 0
        sma30 = float(row['sma30']) + 0
        price = float(row['open']) + 0
        # Check for entry rule and existing position
        # Enter only if you have no existing position
        if (sma7 > sma30) and (btc.l == 0):            
            qty = int(capital/price)
            btc.add_trade(row['date'], 'BTC', price, qty, 'B')
            
        # Check for exit
        if btc.positions['BTC'] > 0:
            qty = btc.positions['BTC'] # Get the present position
            if sma7 < sma30:
                btc.add_trade(row['date'], 'BTC', price , qty, 'S')

In [ ]:
btc, btc.values

**Hurray!** You have made a profit and still hold a position. But its not surprising since bitcoin has increased twenty fold during this period. Let's do some analytics for fun.

Beware, you are not taking commission and transaction costs into account

In [ ]:
trades = pd.DataFrame(btc.all_trades)
trades['ts'] = pd.to_datetime(trades['ts'])
trades['year'] = trades['ts'].dt.year
trades['values'] = trades['qty'] * trades['price']
trades.groupby(['year', 'order']).agg({'qty': sum, 'values': sum}).unstack()

Looks, 2013 and 2017 seemed to be really good years